In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/database-sensor-iklim-litbang-firebase-admin.json")
firebase_admin.initialize_app(source_cred, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-05/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di jalur /auto_weather_stat/id-03/data.")

                 dew  humidity  pressure  rainfall  rainrate  temperature  \
1742366497  27.74858  97.23300  1011.023   11.7348       0.0     28.23109   
1742366512  27.58560  96.49294  1011.033   11.7348       0.0     28.19905   
1742366527  27.56910  96.34035  1011.045   11.7348       0.0     28.20973   
1742366542  27.68267  96.81529  1011.038   11.7348       0.0     28.23911   
1742366557  27.71058  96.92783  1011.046   11.7348       0.0     28.24712   

             timestamp     volt  
1742366497  1742366497  4.20625  
1742366512  1742366512  4.22000  
1742366527  1742366527  4.21125  
1742366542  1742366542  4.21875  
1742366557  1742366557  4.22000  


In [4]:
df.head(10)

,dew,humidity,pressure,rainfall,rainrate,temperature,timestamp,volt
1742366497,27.74858,97.23300,1011.023,11.7348,0.0,28.23109,1742366497,4.20625
1742366512,27.58560,96.49294,1011.033,11.7348,0.0,28.19905,1742366512,4.22000
1742366527,27.56910,96.34035,1011.045,11.7348,0.0,28.20973,1742366527,4.21125
1742366542,27.68267,96.81529,1011.038,11.7348,0.0,28.23911,1742366542,4.21875
1742366557,27.71058,96.92783,1011.046,11.7348,0.0,28.24712,1742366557,4.22000
1742366572,27.70560,96.80957,1011.059,11.7348,0.0,28.26315,1742366572,4.22250
1742366587,27.63344,96.49294,1011.047,11.7348,0.0,28.24712,1742366587,4.22125
1742366602,27.46198,95.62127,1011.058,11.7348,0.0,28.23109,1742366602,4.21750
1742366617,27.59813,96.20493,1011.048,11.7348,0.0,28.26315,1742366617,4.22000
1742366632,27.58824,96.22400,1011.041,11.7348,0.0,28.24979,1742366632,4.21750


In [5]:
# Input readable date
readable_date = "01-03-2025 11:15:00"  # Format: DD-MM-YYYY HH:MM:SS

# Convert readable date to Unix timestamp
start_timestamp = int(datetime.strptime(readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(start_timestamp)
end_timestamp = int(datetime.now().timestamp())  # Current time in Unix timestamp

# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}

1740827700


In [6]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [7]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-05/data', app=dest_app)

In [8]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>